In [9]:
import sys
sys.path.append('..')
from experiments import perform_gradcam, perform_lrp_captum
from internal_utils import preprocess_images, condense_to_heatmap, blur_image_batch, add_random_noise_batch, get_data_imagenette, get_teacher_model, get_CIFAR10_dataloader
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
import torch.nn.functional as F
import torchvision.transforms.functional as TF


In [10]:
from experiments import WrapperNet, WrapperNetContrastive
import torch
from internal_utils import update_dictionary_patch
from baselines.trainVggBaselineForCIFAR10.vgg import vgg11

def get_teacher_model(teacher_checkpoint_path):
    checkpoint = torch.load(teacher_checkpoint_path)
    # assume teacher model is vgg11 for now
    teacher = vgg11()
    try: 
        checkpoint = update_dictionary_patch(checkpoint)
        teacher.load_state_dict(checkpoint['new_state_dict'])
    except:
        print('Incorrect patch specified')
    return teacher
# data = get_CIFAR10_dataloader(train=True, batch_size=8)
# data_test = get_CIFAR10_dataloader(train=False, batch_size=8)
# input_images, labels = next(iter(data))
# teacher_model = WrapperNet(get_teacher_model("/home/charleshiggins/RL-LRP/baselines/trainVggBaselineForCIFAR10/save_vgg11/checkpoint_299.tar"), hybrid_loss=True)
# # define params
# learner_model = WrapperNet(vgg11(), hybrid_loss=True)

In [11]:
# from experiments import perform_lrp_plain
# input_images, labels = next(iter(data))
# sample_image, sample_label = input_images[0], labels[0]
# visualise_panel_image(sample_image.unsqueeze(0), teacher_model, 3, 15, 0.1, 0.5, perform_lrp_plain, sample_label.unsqueeze(0))

In [12]:
# visualise_panel_image(sample_image.unsqueeze(0), learner_model, 3, 15, 0.1, 0.5, perform_lrp_plain, sample_label.unsqueeze(0))

In [13]:
# pp_images = preprocess_images(input_images)
# print(f"The target tensor should be: {labels}")
# output, heatmap = learner_model(pp_images)
# output_target, heatmap_target = teacher_model(pp_images)

In [14]:
import torch.nn as nn
import numpy as np
class CosineDistanceLoss(torch.nn.Module):
    def __init__(self):
        super(CosineDistanceLoss, self).__init__()

    def forward(self, input1, input2):
        # Flatten the images: shape from [b, 1, 28, 28] to [b, 784]
        input1_flat = input1.view(input1.size(0), -1)
        input2_flat = input2.view(input2.size(0), -1)
        
        # Compute cosine similarity, then convert to cosine distance
        cosine_sim = F.cosine_similarity(input1_flat, input2_flat)
        cosine_dist = 1 - cosine_sim
        
        # Calculate the mean of the cosine distances
        loss = cosine_dist.mean()
        # loss = F.mse_loss(input1_flat, input2_flat)
        return loss
    

# Define SSIM loss (we'll minimize 1 - SSIM)
class SSIMLoss(torch.nn.Module):
    def __init__(self, data_range=1.0, size_average=True, channel=3):
        super(SSIMLoss, self).__init__()
        self.ssim_module = SSIM(data_range=data_range, size_average=size_average, channel=channel)
    
    def forward(self, img1, img2):
        ssim_value = self.ssim_module(img1, img2)
        return 1 - ssim_value
    
class Tracker:
    # small tracker class to compute moving averages
    def __init__(self):
        self.ce_losses = []
        self.ssim_losses = []
        self.accs = []
    
    def update(self, ce_loss, ssim_loss, acc):
        # add values to the tracker
        self.ce_losses.append(ce_loss)
        self.ssim_losses.append(ssim_loss)
        self.accs.append(acc)
    
    
    def get_avg(self):
        # get the average of the last 10 values
        if len(self.ce_losses) <= 10:
            return np.mean(self.ce_losses), np.mean(self.ssim_losses), np.mean(self.accs)
        else: 
            return np.mean(self.ce_losses[-10:]), np.mean(self.ssim_losses[-10:]), np.mean(self.accs[-10:])
    
    def get_results(self):
        results = {
            "CrossEntropyLoss": np.array(self.ce_losses),
            "SSIMLoss": np.array(self.ssim_losses),
            "Accuracy": np.array(self.accs)
        }
        return results

            
    
def remove_grad_for_all_but_last_layer(learner_model, optimizer, scheduler, verbose=False):
    for name, module in learner_model.model.named_modules():
        if not isinstance(module, nn.Sequential) \
        and not isinstance(module, WrapperNet) \
        and not len(list(module.children())) > 0 \
            and type(module) not in [nn.ReLU, nn.MaxPool2d, nn.AdaptiveAvgPool2d, nn.LogSoftmax, nn.Dropout]:
            if verbose:
                print(name)
                print(module)
                print("####################### \n")
            if "classifier.6" not in name:
                print(f"removing grad from: {name}")
                for param in module.parameters():
                    param.requires_grad = False
            else:
                print(f"Grad will continue for {name}")
    optimizer = torch.optim.SGD(learner_model.parameters(), lr=0.5, momentum=0.9, weight_decay=5e-4)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)

    return learner_model, optimizer, scheduler



In [18]:
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
import numpy as np
# get the dataset
data = get_CIFAR10_dataloader(train=True, batch_size=8)
data_test = get_CIFAR10_dataloader(train=False, batch_size=8)
# load data and preprocess
input_images, labels = next(iter(data))
pp_images = preprocess_images(input_images)
# break to avoiding having to reload the data everytime

def train_baseline(pp_images, labels, teacher_heatmap, optimizer, scheduler, epochs=100):
    # we pass in the optimizer and scheduler to allow for the learning rate to be updated and changed
    # to keep things the same
    learner_model = WrapperNet(vgg11(), hybrid_loss=True)
    ssim_loss = SSIMLoss()
    cross_entropy_loss = torch.nn.CrossEntropyLoss()
    tracker = Tracker()
    for i in range(epochs):
        model_out, model_heatmap = learner_model(pp_images)
        # loss for measurement, but not for backprop
        img_loss = ssim_loss(model_heatmap, teacher_heatmap)
        # now training loop starts
        optimizer.zero_grad()
        acc_loss = cross_entropy_loss(torch.nn.functional.log_softmax(model_out, dim=1), labels)
        acc_loss.backward()
        torch.nn.utils.clip_grad_norm_(learner_model.parameters(), 1)
        optimizer.step()
        scheduler.step()
        # store results
        correct = model_out.argmax(dim=1).eq(labels).sum().item()
        correct_pct = 100 * correct/labels.shape[0] 
        tracker.update(ce_loss = acc_loss.item(), ssim_loss = img_loss.item(), acc = correct_pct)
        mov_ce, mov_ssim, mov_acc = tracker.get_avg()
        print(f"iteration: {i} \t accuracy: {correct_pct:.4f} ({mov_acc:.4f})\t image loss: {img_loss.float():.4f} ({mov_ssim:.4f})\t cross entropy loss: {acc_loss.float():.4f} ({mov_ce:.4f}) \t learning rate: {optimizer.param_groups[0]['lr']:.4f}")
        # clean up hooks
        learner_model.remove_hooks()
        learner_model.reapply_hooks()
    results = tracker.get_results()
    print("#" * 10)
    print("#" * 10)
    print("Baseline training completed")
    print("#" * 10)
    print("#" * 10)
    return results, learner_model


## The following code block starts by training to optimise solely the heatmap reconstruction loss. 
The idea belying this is to use the heatmap training loss to directly optimize the parameters of the underlying learner network to pay attention to specific features. While this can lead to a slight increase in accuracy over the cross entropy loss, it is considerably slower than regular training. We demonstrate that after considerable "pre-training" post-training achieves a high level of performance faster than simply training from scratch.  

In [20]:

import copy
# pp_images, labels = pp_images[:3], labels[:3]
# define losses
mse_loss = torch.nn.MSELoss()
cos_loss = CosineDistanceLoss()
cross_entropy_loss = torch.nn.CrossEntropyLoss()
ssim_loss = SSIMLoss()
# get clean models
teacher_model = WrapperNet(get_teacher_model("/home/charleshiggins/RL-LRP/baselines/trainVggBaselineForCIFAR10/save_vgg11/checkpoint_299.tar"), hybrid_loss=True)
learner_model = WrapperNet(vgg11(), hybrid_loss=True)
# define optimizers
optimizer = torch.optim.SGD(learner_model.parameters(), lr=0.5, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

CHANGE_POINT = 100
EPOCHS = 400
tracker = Tracker()

print("target labels: ", labels)
with torch.no_grad():
    _, teacher_heatmap = teacher_model(pp_images, labels)

# make copy of scheduler and optimizer to pass into the baseline
optimizer_copy = copy.deepcopy(optimizer)
scheduler_copy = torch.optim.lr_scheduler.StepLR(optimizer_copy, step_size=50, gamma=0.9)
# train a baseline for regulat performance -- this is a model trained only on cross-entropy loss
baseline_results, baseline_model = train_baseline(pp_images, labels, teacher_heatmap, optimizer_copy, scheduler_copy, epochs=EPOCHS)

############################################
# Now we will train the model with only the SSIM loss
############################################
for i in range(0, CHANGE_POINT):
    model_out, model_heatmap = learner_model(pp_images, labels)
    # loss for measurement, but not for backprop
    acc_loss = cross_entropy_loss(torch.nn.functional.log_softmax(model_out, dim=1), labels)
    # now training loop starts
    optimizer.zero_grad()
    img_loss = ssim_loss(model_heatmap, teacher_heatmap)
    img_loss.backward()
    torch.nn.utils.clip_grad_norm_(learner_model.parameters(), 1)
    optimizer.step()
    scheduler.step()
    # store results
    correct = model_out.argmax(dim=1).eq(labels).sum().item()
    correct_pct = 100 * correct/labels.shape[0] 
    tracker.update(ce_loss = acc_loss.item(), ssim_loss = img_loss.item(), acc = correct_pct)
    mov_ce, mov_ssim, mov_acc = tracker.get_avg()
    print(f"iteration: {i} \t accuracy: {correct_pct:.4f} ({mov_acc:.4f})\t image loss: {img_loss.float():.4f} ({mov_ssim:.4f})\t cross entropy loss: {acc_loss.float():.4f} ({mov_ce:.4f}) \t learning rate: {optimizer.param_groups[0]['lr']:.4f}")
    # clean up hooks
    learner_model.remove_hooks()
    learner_model.reapply_hooks()

# Write logic now to update or change the model/parameters/optimizers 
    
# training loop after change
print("############" * 4)
print("## Training loop after change ##")
print("############" * 4)
for i in range(CHANGE_POINT, EPOCHS):
    model_out, model_heatmap = learner_model(pp_images)
    # loss for measurement, but not for backprop
    img_loss = ssim_loss(model_heatmap, teacher_heatmap)
    # now training loop starts
    optimizer.zero_grad()
    acc_loss = cross_entropy_loss(torch.nn.functional.log_softmax(model_out, dim=1), labels)
    acc_loss.backward()
    torch.nn.utils.clip_grad_norm_(learner_model.parameters(), 1)
    optimizer.step()
    # scheduler.step()
    # store results
    correct = model_out.argmax(dim=1).eq(labels).sum().item()
    correct_pct = 100 * correct/labels.shape[0] 
    tracker.update(ce_loss = acc_loss.item(), ssim_loss = img_loss.item(), acc = correct_pct)
    mov_ce, mov_ssim, mov_acc = tracker.get_avg()
    print(f"iteration: {i} \t accuracy: {correct_pct:.4f} ({mov_acc:.4f})\t image loss: {img_loss.float():.4f} ({mov_ssim:.4f})\t cross entropy loss: {acc_loss.float():.4f} ({mov_ce:.4f}) \t learning rate: {optimizer.param_groups[0]['lr']:.4f}")
    # clean up hooks
    learner_model.remove_hooks()
    teacher_model.remove_hooks()
    learner_model.reapply_hooks()
    teacher_model.reapply_hooks()
              

/var/tmp/ipykernel_33058/4006691909.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(teacher_checkpoint_path)


target labels:  tensor([6, 9, 9, 4, 1, 1, 2, 7])
iteration: 0 	 accuracy: 0.0000 (0.0000)	 image loss: 0.6274 (0.6274)	 cross entropy loss: 2.3054 (2.3054) 	 learning rate: 0.5000
iteration: 1 	 accuracy: 0.0000 (0.0000)	 image loss: 0.5953 (0.6114)	 cross entropy loss: 2.3030 (2.3042) 	 learning rate: 0.5000
iteration: 2 	 accuracy: 12.5000 (4.1667)	 image loss: 0.6037 (0.6088)	 cross entropy loss: 2.3063 (2.3049) 	 learning rate: 0.5000
iteration: 3 	 accuracy: 12.5000 (6.2500)	 image loss: 0.6087 (0.6088)	 cross entropy loss: 2.2986 (2.3034) 	 learning rate: 0.5000
iteration: 4 	 accuracy: 12.5000 (7.5000)	 image loss: 0.6250 (0.6120)	 cross entropy loss: 2.3021 (2.3031) 	 learning rate: 0.5000
iteration: 5 	 accuracy: 12.5000 (8.3333)	 image loss: 0.6098 (0.6117)	 cross entropy loss: 2.2954 (2.3018) 	 learning rate: 0.5000
iteration: 6 	 accuracy: 12.5000 (8.9286)	 image loss: 0.6227 (0.6132)	 cross entropy loss: 2.2998 (2.3015) 	 learning rate: 0.5000
iteration: 7 	 accuracy: 25.0

KeyboardInterrupt: 